In [1]:
%pip install scrapy 
%pip install crochet 
%pip install scrapy-selenium
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ------ --------------------------------- 10.2/59.7 kB ? eta -:--:--
     ------ --------------------------------- 10.2/59.7 kB ? eta -:--:--
     -------------------------------------- 59.7/59.7 kB 396.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.2/11.5 MB 6.1 MB/s eta 0:00:02
   - -------------------------------------- 0.4/11.5 MB 5.4 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/11.5 MB 6.0 MB/s eta 0:00:02
   ---- ----------------------------------- 1.2/11.5 MB 7.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.5 MB 8.2 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.5 MB 8.0 MB/s eta 0:00:02
   --------- ------------------------------ 2.7/11.5 MB 8.5 MB/s eta 0:00:02
   ---------- -------

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\f2py.exe' -> 'c:\\Python312\\Scripts\\f2py.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from typing import Iterable
import scrapy
from scrapy_selenium import SeleniumRequest


class itemFormat(scrapy.Item):

    card_name = scrapy.Field()
    creature_type = scrapy.Field()
    mana_cost = scrapy.Field()
    card_text = scrapy.Field()
    legendary = scrapy.Field()
    card_type = scrapy.Field()

class databaseSpider(scrapy.Spider):
    name = "databaseSpider"

    custom_settings = {
        'FEED_FORMAT':'json',
        'FEED_URI':f'databaseSpider.json',
    }

    def start_requests(self) -> Iterable[scrapy.Request]:
        base_url = ['https://gatherer.wizards.com/Pages/Search/Default.aspx?color=|[W]|[U]|[B]|[R]|[G]']

        for url in base_url:
            yield SeleniumRequest(url = url, callback = self.parse)
        return super().start_requests()

    def parse(self, response):

        for i, card in enumerate(response.css('tr.cardItem.oddItem')):
            type_check = card.css('div.cardInfo span.typeLine::text').get(i)
            card_item = itemFormat()
            

            card_item['card_name'] = card.css('div.cardInfo span.cardTitle a::text').get(i)

            if "Creature" in type_check:

                if "\u2014" in card.css('div.cardInfo span.typeLine::text').get(i):
                    text = card.css('div.cardInfo span.typeLine::text').get(i).replace("\u2014"," ").replace("                                 "," ").replace("\r\n"," ").replace("                                 "," ").replace("Creature", " ").replace("      "," ")
                    card_item["card_type"] = "Creature"
                    card_item['creature_type'] = text

                    if "Legendary" in text:
                        card_item['legendary'] = True
                    else:
                        card_item['legendary'] = False
                        
                else:
                    card_item['card_type'] = card.css('div.cardInfo span.typeLine::text').get(i).replace("                                 "," ").replace("\r\n"," ").replace("                                 "," ")
            
            elif "Enchantment" in type_check:
                card_item['card_type'] = "Enchantment"
            elif "Aura" in type_check:
                card_item['card_type'] = "Aura"
            elif "Instant" in type_check:
                card_item['card_type'] = "Instant"
            elif "Battle" in type_check:
                card_item['card_type'] = "Battle"
            elif "Sorcery" in type_check:
                card_item['card_type'] = "Sorcery"
        

            card_item['card_text'] = card.css('div.rulesText p::text').get(i)
            card_item['mana_cost'] = card.css('span.manaCost::attr(alt)').get(i)

            yield card_item

In [3]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
    
process.crawl(databaseSpider)
process.start()

2024-08-19 15:22:47 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-08-19 15:22:47 [scrapy.utils.log] INFO: Versions: lxml 5.2.2.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.0, Twisted 24.7.0, Python 3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.1 4 Jun 2024), cryptography 43.0.0, Platform Windows-10-10.0.19045-SP0
2024-08-19 15:22:47 [scrapy.addons] INFO: Enabled addons:
[]
2024-08-19 15:22:47 [py.warnings] WARNING: c:\Python312\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FIN

In [7]:
import pandas

file_name = f'databaseSpider.json'
dfjson = pandas.read_json(file_name)
dfjson

ModuleNotFoundError: No module named 'pandas'